In [12]:
import pandas as pd

TRUTH_TABLE_2 = pd.DataFrame([
    [str(x) for x in [0,0,0]],
    [str(x) for x in [0,1,0]],
    [str(x) for x in [1,0,0]],
    [str(x) for x in [1,1,1]],
])

TRUTH_TABLE_2.columns = ['A','B','Y']

TRUTH_TABLE_3 = pd.DataFrame([
    [str(x) for x in [0,0,0,1]],
    [str(x) for x in [0,0,1,0]],
    [str(x) for x in [0,1,0,1]],
    [str(x) for x in [0,1,1,0]],
    [str(x) for x in [1,0,0,1]],
    [str(x) for x in [1,0,1,0]],
    [str(x) for x in [1,1,0,1]],
    [str(x) for x in [1,1,1,1]]
])

TRUTH_TABLE_3.columns = ['A','B','C','Y']

TRUTH_TABLE_4 = pd.DataFrame([
    [str(a), str(b), str(c), str(d), 
     str((a and b) or (c and d))]   # function: Y = AB + CD
    for a in [0,1]
    for b in [0,1]
    for c in [0,1]
    for d in [0,1]
])

TRUTH_TABLE_4.columns = ['A','B','C','D','Y']
TRUTH_TABLE_4

,A,B,C,D,Y
0,0,0,0,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,0,1,1,1
4,0,1,0,0,0
5,0,1,0,1,0
6,0,1,1,0,0
7,0,1,1,1,1
8,1,0,0,0,0
9,1,0,0,1,0


In [ ]:
def can_merge(groups):
    cm = True
    for group in groups:
        pass

def merge_vals(group1, group2):
    mt1, val1 = group1
    mt2, val2 = group2

    diff = 0
    merge = True
    diffindex = -1
    for c1, c2 in zip(val1, val2):
        if c1!=c2:
            diff += 1
            diffindex = val1.index(c1)
            if diff > 1:
                merge = False
                break
    
    if not merge:
        return None

    mt = list(set(mt1+mt2))
    val = val1
    val[diffindex] = '-'
    return (mt,val)


def merge_groups(groups):
    if not can_merge(groups):
        return groups
    else:
        new_groups = []
        n = len(groups)
        for i in range(n-1):
            g = []
            curr_group = groups[i]
            next_group = groups[i+1]
            for g1 in curr_group:
                for g2 in next_group:
                    mg = merge_vals(g1,g2)
                    if mg:
                        g.append(mg)
            new_groups.append(g)
        
        return merge_groups(new_groups)

def create_implicant_table(groups):
    minterms = set()
    for group in groups:
        for mt,_ in group:
            minterms.add(x for x in mt)
    
    

def QMC(TRUTH_TABLE):
    M = TRUTH_TABLE[TRUTH_TABLE['Y']=='1'].drop(['Y'],axis=1)
    m,n = M.shape
    groups = []
    curr_1s = -1
    for i in range(m):
        group_mins = [M.index[i]]
        group_value = ""
        ones = 0
        for j in range(n):
            group_value += M.iloc[i,j]
            if(M.iloc[i,j]=='1'):
                ones += 1
        
        if ones>curr_1s:
            groups.append([(group_mins,group_value)])
            curr_1s = ones
        else:
            groups[-1].append((group_mins,group_value))

    groups = merge_groups(groups)


    return M.index

QMC(TRUTH_TABLE_2)

Index([3], dtype='int64')

In [5]:
def make_minterms(TRUTH_TABLE):
    M = TRUTH_TABLE[TRUTH_TABLE['Y']=='1'].drop(['Y'],axis=1)
    m,n = M.shape
    minterms = []
    for i in range(m):
        mt = ""
        ones = 0
        for j in range(n):
            mt += M.iloc[i,j]
            ones += 1
        if ones>0:
            minterms.append(mt)
    return minterms

In [ ]:
def get_prime_implicants(minterms):
    prime_implicants = []
    merges = [False for _ in range(len(minterms))]
    n_merges = 0
    merged_mt, mt1, mt2 = "", "", ""

    for i in range(len(minterms)):
        for c in range(i+1,len(minterms)):
            mt1 = minterms[i]
            mt2 = minterms[c]
            if dashes_align(mt1, mt2) and one_bit_diff_no_dash(mt1, mt2):
                merged_mt = merge_minterms(mt1, mt2)
                if merged_mt not in prime_implicants:
                    prime_implicants.append(merged_mt)
                n_merges += 1
                merges[i] = True
                merges[c] = True

    for j in range(len(minterms)):
        if merges[j]==False and minterms[j] not in prime_implicants:
            prime_implicants.append(minterms[j])
        
    if n_merges==0:
        return prime_implicants
    else:
        return get_prime_implicants(prime_implicants)
    
def one_bit_diff_no_dash(a: str, b: str) -> bool:
  diff = 0
  for c1, c2 in zip(a, b):
      if c1 != c2:
          if c1 == '-' or c2 == '-':
              return False
          diff += 1
          if diff > 1:
              return False
  return diff == 1

def merge_minterms(a: str, b: str) -> str | None:
  if not one_bit_diff_no_dash(a, b):
      return None
  # exactly one real-bit difference → put '-' there
  out = []
  for c1, c2 in zip(a, b):
      out.append('-' if c1 != c2 else c1)
  return ''.join(out)


def dashes_align(mt1,mt2):
    for i in range(len(mt1)):
        if mt1[i]!='-' and mt2[i]=='-':
            return False
    
    return True

def int_expr(mt):
    i = 0
    for c in mt:
        i *= 10
        if c=='-':
            continue
        i += (c-'0')
    return i

def create_implicant_chart(prime_implicants, minterms):
    chart = {}
    for pi in prime_implicants:
        row = "".join("1" if covers(pi, m) else "0" for m in minterms)
        chart[pi] = row
    return chart


def covers(mask: str, mint: str) -> bool:
    return all(mc == '-' or mc == xc for mc, xc in zip(mask, mint))

def select_implicants(chart, minterms):
    selected = []
    covered = set()

    # Step 1: essential implicants
    for j, mt in enumerate(minterms):
        covering = [pi for pi, row in chart.items() if row[j] == "1"]
        if len(covering) == 1:
            epi = covering[0]
            if epi not in selected:
                selected.append(epi)
            for k, bit in enumerate(chart[epi]):
                if bit == "1":
                    covered.add(minterms[k])

    # Step 2 + 3: greedy cover
    while len(covered) < len(minterms):
        # find best PI covering most uncovered
        best_pi = max(
            chart.keys(),
            key=lambda pi: sum(1 for k, mt in enumerate(minterms)
                               if chart[pi][k] == "1" and mt not in covered)
        )
        selected.append(best_pi)
        for k, mt in enumerate(minterms):
            if chart[best_pi][k] == "1":
                covered.add(mt)

    return selected



['--11', '11--']